In [ ]:
#common packages we basically always need
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
#load the MNIST dataset with binary pixel values
import scipy.io as sio
data = sio.loadmat('mnist.mat')
print(data)



Xtrain, Xtest = data['trainX'].astype(float), data['testX'].astype(float)
ytrain, ytest = data['trainY'][0], data['testY'][0]

#pull and plot some samples
for k in range(9):
    plot_data = Xtrain[k,:]
    plot_data = np.reshape(plot_data,(28,28))
    plot_label = ytrain[k]
    plt.subplot(3,3,k+1)
    plt.imshow(plot_data)
    plt.title(plot_label)
plt.tight_layout()

def get_small_dataset(X,y,m):
    return X[:m,:],y[:m]


num_labels = len(np.unique(ytrain))
num_feats = Xtrain.shape[1]

In [3]:

def get_dist(Xtrain,zquery):
    term1 = np.sum(Xtrain*Xtrain,axis=1)
    term2 = np.dot(Xtrain,zquery)
    term3 = np.dot(zquery,zquery)
    return term1 - 2*term2 + term3
    
print(get_dist(Xtrain,Xtrain[0,:])[0])
print(get_dist(Xtrain,Xtest[0,:])[10])
print(get_dist(Xtrain,Xtest[10,:])[50])




0.0
6069462.0
5661744.0


In [4]:
import scipy.stats as ss

m = 100
K = 3

Xtrain_small, ytrain_small = get_small_dataset(Xtrain,ytrain,m)

def pred(zquery,Xtrain,ytrain, K):
    d = get_dist(Xtrain,zquery)
    idx = np.argsort(d)
    labels = ytrain[idx[:K]]
    return ss.mode(labels)[0][0]
    
ytest_pred = ytest + 0    
for k in range(Xtest.shape[0]):
    z = Xtest[k,:]
    ytest_pred[k] = pred(z,Xtrain_small, ytrain_small, K)
    
print(ytest_pred[:20])
print(ytest[:20])

[7 2 1 0 4 1 4 4 6 9 0 0 9 0 1 9 7 7 3 4]
[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]


In [5]:
def get_accuracy(ytest, ypred):
    return np.mean(np.equal(ytest,ypred))

get_accuracy(ytest,ytest_pred)

0.6476

In [6]:
import time
for m in [100,1000, 2500]:
    Xtrain_small, ytrain_small = get_small_dataset(Xtrain,ytrain,m)
    for K in [1,3,5]:
        
        start = time.time()
        ytest_pred = ytest + 0    
        for k in range(Xtest.shape[0]):
            z = Xtest[k,:]
            ytest_pred[k] = pred(z,Xtrain_small, ytrain_small, K)
    
        print(m,K,get_accuracy(ytest,ytest_pred), time.time()-start)

100 1 0.6794 3.055835008621216
100 3 0.6476 3.128586530685425
100 5 0.6232 3.1971077919006348
1000 1 0.869 47.93265891075134
1000 3 0.8622 49.253700256347656
1000 5 0.8582 47.04932427406311
2500 1 0.9136 118.01626539230347
2500 3 0.9146 122.57662630081177
2500 5 0.9101 125.25599884986877
